In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

def load_kat_mx_file(file_path):
    """Load .mx data into numpy array; extract k from comment line."""
    with open(file_path, 'r') as f:
        lines = []
        k = None
        for line in f:
            if line.startswith('# Kmer value:'):
                k = int(line.strip().split(':')[1])
            if not line.startswith('#'):
                lines.append(line.strip())
        matrix_data = [list(map(float, line.split())) for line in lines]
        matrix = np.array(matrix_data)
    return matrix, k 

def compute_metrics(matrix, k):
    """
    Compute metrics from the matrix:
    - singleton_rdna_kmers: Number of kmers in rDNA with count 1
    - distinct_rdna_kmers: Number of distinct kmers in rDNA(count > 0)
    - total_rdna_kmers: Total number of kmers in rDNA
    - unique_rdna_kmers: Number of kmers in rDNA not occurring in reference genome
    - high_freq_unique_kmers: Number of unique kmers in rDNA with count >= 3 not occurring in reference genome
    - shared_kmers: Number of kmers shared between rDNA and reference genome
    - specificity: Ratio of distinct kmers in rDNA to total kmers in rDNA
    """
    metrics = {
        'k': k,
        'singleton_rdna_kmers': int(np.sum(matrix[:,1])),
        'distinct_rdna_kmers': int(np.sum(matrix[:,1:])),
        'total_rdna_kmers': int(np.sum(matrix[:, 1:] * np.arange(1, matrix.shape[1]))),
        'unique_rdna_kmers': int(np.sum(matrix[0,:])),
        'high_freq_unique_kmers': int(np.sum(matrix[0, 3:])),
        'shared_kmers': int(np.sum(matrix[1:, 1])),
        'precision': np.sum(matrix[:,1:]) / np.sum(matrix[:, 1:] * np.arange(1, matrix.shape[1]))

    }
    return metrics


In [29]:
matdir = '/d/hd09/zhuohan/ectopic_rdna_proj/results/katcomp_matrices_hg002/'
mx_files = glob.glob(os.path.join(matdir, '*.mx'))
all_metrics = []

for mx_file in mx_files:
    matrix, k = load_kat_mx_file(mx_file)
    metrics = compute_metrics(matrix, k)
    all_metrics.append(metrics)

df = pd.DataFrame(all_metrics)
df.sort_values(by='k', inplace=True)
display(df)

,k,singleton_rdna_kmers,distinct_rdna_kmers,total_rdna_kmers,unique_rdna_kmers,high_freq_unique_kmers,shared_kmers,precision
1,15,33260,36223,42839,5729,58,27751,0.845561
14,17,34266,37119,42819,15484,115,19404,0.866882
0,19,35112,37855,42799,21547,130,14441,0.884483
15,21,35868,38470,42779,24535,137,12335,0.899273
2,23,36520,38970,42760,26639,147,10971,0.911366
16,25,37069,39377,42742,28390,154,9856,0.921272
3,27,37542,39711,42724,29930,151,8858,0.929478
17,29,37939,39985,42706,31349,146,7897,0.936285
12,31,38289,40219,42688,32601,135,7022,0.942162
7,33,38608,40426,42670,33690,120,6253,0.947410


In [30]:
df.to_csv('/d/hd09/zhuohan/ectopic_rdna_proj/results/katcomp_metrics_hg002.csv', index=False)